# **PART - I**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('dataset.csv')

In [ ]:
df.dtypes

In [ ]:
df.head(50)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
  df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
df=df.dropna()

In [ ]:
df.describe()

In [ ]:
plt.plot(df['f1'],df['target'], alpha=0.5, color='green')
plt.xlabel('f1')
plt.ylabel('target')
plt.show()

In [ ]:
plt.scatter(df['f2'],df['target'], alpha=0.5, color='green')
plt.xlabel('f2')
plt.ylabel('target')
plt.show()

In [ ]:
plt.scatter(df['f3'],df['target'], alpha=0.5, color='green')
plt.xlabel('f3')
plt.ylabel('target')
plt.show()

In [ ]:
pip install scikit-learn


In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df= scaler.fit_transform(df)
def normalize_df(df,col):
  mini = df[col].min()
  maxi = df[col].max()
  df[col]= df[col].apply(lambda x: (x-mini)/(maxi-mini))
  return df

for col in df.columns:
  df = normalize_df(df,col)
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaled_df = scaler.fit_transform(df)
# df = pd.DataFrame(scaled_df, columns=df.columns)

In [ ]:
df

In [ ]:

from sklearn.model_selection import train_test_split
X = df.drop(columns=['target'])
y = df['target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [ ]:
y_train

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
# def xavier_init(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         if m.bias is not None:
#             nn.init.zeros_(m.bias)

In [ ]:
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
# model.apply(xavier_init)

In [ ]:
pip install torchinfo

In [ ]:
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)
X_test = torch.FloatTensor(X_test)
y_train = torch.FloatTensor(y_train)
y_val = torch.FloatTensor(y_val)
y_test = torch.FloatTensor(y_test)

In [ ]:
from torchinfo import summary
batch_size = 32
input_data = torch.randn(batch_size, input_size)
print(summary(model, input_data=input_data))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))

In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
torch.save(model.state_dict(), "part_1_model_75.44.h5")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(epochs), train_losses)
plt.ylabel("Loss")
plt.xlabel("Epoch")

In [ ]:
plt.plot(range(epochs), val_losses)
plt.ylabel("Validation Loss")
plt.xlabel("Epoch")

In [ ]:
plt.plot(range(epochs), test_losses)
plt.ylabel("Test Loss")
plt.xlabel("Epoch")

In [ ]:
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)


In [ ]:
print(y_pred.dtype)
print(y_train.dtype)


In [ ]:
print(y_val.shape)
print(y_val.dtype)
print(y_pred_val.shape)
print(y_pred_val.dtype)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
print(y_pred_val.dtype)
print(y_val.dtype)

In [ ]:
print(y_test.dtype)
print(y_pred_test.shape)
print(y_pred_test.dtype)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(y_pred_test.dtype)
print(y_test.dtype)

In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

In [ ]:
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")

In [ ]:
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


In [ ]:
plt.figure(figsize=(3, 3))
# plt.plot(accuracy*100, label='Training Accuracy')
# plt.plot(accuracy_val*100, label='Validation Accuracy')
# plt.plot(accuracy_test*100, label='Test Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Training, Validation, and Test Accuracy')
# plt.legend()
# plt.grid(True)
# plt.show()
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()

# **PART-II**

### **Dropout-0.1**

In [ ]:
import torch.optim as optim
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.1
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Dropout-0.2**

In [ ]:
import torch.optim as optim
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.2
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()
torch.save(model.state_dict(), "part_2_dropout_model_74.56.h5")


### **Dropout-0.5**

In [ ]:
import torch.optim as optim
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.5
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Optimizer-Adam**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Optimizer-Adagrad**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
base_accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {base_accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, base_accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()
torch.save(model.state_dict(), "part_2_optimizer_model_77.19.h5")



### **Optimizer-Adamax**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Activation Function-tanh**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.tanh(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.tanh(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Activation Function-selu**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.selu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.selu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()



### **Activation Function-elu**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.elu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.elu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()
torch.save(model.state_dict(), "part_2_ActivationFunction_model_77.19.h5")



### **Initializer-Xavir**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
model.apply(xavier_init)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()
torch.save(model.state_dict(), "part_2_Initializer_model_72.81.h5")

### **Kaiming uniform initialization**

In [ ]:
import torch.nn.init as init
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        init.kaiming_uniform_(self.input_layer.weight, mode='fan_in', nonlinearity='relu')
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        for layer in self.hidden_layers:
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='relu')
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        init.kaiming_uniform_(self.output_layer.weight, mode='fan_in', nonlinearity='relu')
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()

### **Initializer-lecun_init**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
def lecun_init(m):
    if isinstance(m, nn.Linear):
        fan_in = m.weight.size(1)
        nn.init.normal_(m.weight, mean=0, std=1 / fan_in)
        nn.init.constant_(m.bias, 0)
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
model.apply(lecun_init)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, thresholds = roc_curve(y_test, y_pred_test)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
plt.figure(figsize=(3, 3))
model_names = ['Train', 'Validation','Test']
accuracies = [accuracy, accuracy_val, accuracy_test]
plt.bar(model_names, accuracies)
plt.ylabel('Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
plt.plot(range(epochs), train_losses, label='Training Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.plot(range(epochs), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()

# **EARLY STOPPING**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
early_stopping_patience = 10
best_val_loss = float("inf")
best_model_state = None
stop_flag = 0
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    end_time = time.time()
    training_time = end_time - start_time
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        stop_flag = 0
    else:
        stop_flag += 1
        if stop_flag >= early_stopping_patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break
print('Finished Training')
if best_model_state is not None:
    model.load_state_dict(best_model_state)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
print(f"Training Time: {training_time:.2f} seconds")
print("Train Accuracy")
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Validation Accuracy")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
print("Test Accuracy")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
plt.figure(figsize=(3, 3))
model_names = ['Base Model', 'Current Model']
test_accuracies = [base_accuracy_test, accuracy_test]
plt.bar(model_names, test_accuracies)
plt.ylabel('Test Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
percentage_change = ((accuracy_test - base_accuracy_test) / base_accuracy_test) * 100
plt.bar(model_names, [0, percentage_change], color=['blue', 'green'])
plt.ylabel('Percentage Change in Test Accuracy')
plt.title('Comparison of Test Accuracy Improvement')
plt.show()

# **LEARNING RATE SCHEDULER**

In [ ]:
from torch.optim import lr_scheduler
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
    scheduler.step()
print('Finished Training')
if best_model_state is not None:
    model.load_state_dict(best_model_state)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
plt.figure(figsize=(3, 3))
model_names = ['Base Model', 'Current Model']
test_accuracies = [base_accuracy_test, accuracy_test]
plt.bar(model_names, test_accuracies)
plt.ylabel('Test Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
percentage_change = ((accuracy_test - base_accuracy_test) / base_accuracy_test) * 100
plt.bar(model_names, [0, percentage_change], color=['blue', 'green'])
plt.ylabel('Percentage Change in Test Accuracy')
plt.title('Comparison of Test Accuracy Improvement')
plt.show()


# **BATCH NORMALIZATION**

In [ ]:
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.bn_input = nn.BatchNorm1d(hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_sizes[i], hidden_sizes[i + 1]) for i in range(len(hidden_sizes) - 1)
        ])
        self.bn_hidden = nn.ModuleList([
            nn.BatchNorm1d(hidden_sizes[i + 1]) for i in range(len(hidden_sizes) - 1)
        ])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.bn_input(x)
        for i, layer in enumerate(self.hidden_layers):
            x = F.relu(layer(x))
            x = self.bn_hidden[i](x)
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
plt.figure(figsize=(3, 3))
model_names = ['Base Model', 'Current Model']
test_accuracies = [base_accuracy_test, accuracy_test]
plt.bar(model_names, test_accuracies)
plt.ylabel('Test Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
percentage_change = ((accuracy_test - base_accuracy_test) / base_accuracy_test) * 100
plt.bar(model_names, [0, percentage_change], color=['blue', 'green'])
plt.ylabel('Percentage Change in Test Accuracy')
plt.title('Comparison of Test Accuracy Improvement')
plt.show()


# **GRADIENT CLIPPING**


In [ ]:
import torch.optim as optim
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
from sklearn.model_selection import KFold
torch.manual_seed(42)
class NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
model = NN_Model(input_size, hidden_sizes, output_size,dropout_prob)
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
validation_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()
print("Unique values in y_train:", torch.unique(y_train))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)
epochs = 100
train_losses=[]
val_losses=[]
test_losses=[]
max_gradient_norm = 1.0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(trainloader)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(validationloader, 0):
            y_pred_val = model.forward(X_val)
            val_loss = criterion(y_pred_val, y_val)
            val_loss += val_loss.item()
    avg_val_loss = val_loss / len(validationloader)
    train_losses.append(val_loss.detach().numpy())
    val_losses.append(avg_val_loss.detach().numpy())
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
      for i, data in enumerate(testloader, 0):
        y_pred_test = model.forward(X_test)
        test_loss = criterion(y_pred_test, y_test)
        test_loss += test_loss.item()
    avg_test_loss = test_loss/len(testloader)
    test_losses.append(avg_test_loss.detach().numpy())
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}, Test Loss: {avg_test_loss}')
print('Finished Training')
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
y_pred_val = np.argmax(y_pred_val.detach().numpy(), axis=1)
y_pred_test=np.argmax(y_pred_test.detach().numpy(), axis=1)
accuracy = accuracy_score(y_train, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, y_pred, average='binary')
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val, recall_val, f1_val, _ = precision_recall_fscore_support(y_val, y_pred_val, average='binary')
print(f"Accuracy: {accuracy_val * 100:.2f}%")
print(f"Precision: {precision_val:.2f}")
print(f"Recall: {recall_val:.2f}")
print(f"F1 Score: {f1_val:.2f}")
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Accuracy: {accuracy_test * 100:.2f}%")
print(f"Precision: {precision_test:.2f}")
print(f"Recall: {recall_test:.2f}")
print(f"F1 Score: {f1_test:.2f}")
plt.figure(figsize=(3, 3))
model_names = ['Base Model', 'Current Model']
test_accuracies = [base_accuracy_test, accuracy_test]
plt.bar(model_names, test_accuracies)
plt.ylabel('Test Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
percentage_change = ((accuracy_test - base_accuracy_test) / base_accuracy_test) * 100
plt.bar(model_names, [0, percentage_change], color=['blue', 'green'])
plt.ylabel('Percentage Change in Test Accuracy')
plt.title('Comparison of Test Accuracy Improvement')
plt.show()


# **K-FOLD, ENSEMBLE MODEL**

In [ ]:
import torch.optim as optim
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
from sklearn.model_selection import KFold
torch.manual_seed(42)
class Improved_NN_Model(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size,dropout_prob):
        super(Improved_NN_Model, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes) - 1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size, dtype=torch.float)
        self.dropout = nn.Dropout(dropout_prob)
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x
input_size = 7
hidden_sizes = [128, 128, 128]
output_size = 2
dropout_prob = 0.3
num_models = 5
ensemble_models = []
kf = KFold(n_splits=num_models)
kf.get_n_splits(X)
for fold, (train_indices, test_indices) in enumerate(kf.split(X)):
    K_X_train, K_X_test = X.iloc[train_indices].to_numpy(), X.iloc[test_indices].to_numpy()
    K_y_train, K_y_test = y.iloc[train_indices].to_numpy(), y.iloc[test_indices].to_numpy()
    K_train_dataset = TensorDataset(torch.FloatTensor(K_X_train), torch.LongTensor(K_y_train))
    K_test_dataset = TensorDataset(torch.FloatTensor(K_X_test), torch.LongTensor(K_y_test))
    K_trainloader = DataLoader(K_train_dataset, batch_size=batch_size, shuffle=True)
    K_testloader = DataLoader(K_test_dataset, batch_size=batch_size)
    # K_train_dataset = TensorDataset(K_X_train, K_y_train)
    # K_test_dataset = TensorDataset(K_X_test, K_y_test)
    # K_trainloader = DataLoader(K_train_dataset, batch_size=batch_size, shuffle=True)
    # K_testloader = DataLoader(K_test_dataset, batch_size=batch_size)
    model = NN_Model(input_size, hidden_sizes, output_size, dropout_prob)
    optimizer = optim.Adagrad(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    epochs = 100
    train_losses = []
    test_losses = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(K_trainloader, 0):
            optimizer.zero_grad()
            K_y_pred = model.forward(torch.FloatTensor(K_X_train))
            loss = criterion(K_y_pred, torch.LongTensor(K_y_train))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(K_testloader, 0):
                K_y_pred_test = model.forward(torch.FloatTensor(K_X_test))
                test_loss = criterion(K_y_pred_test, torch.LongTensor(K_y_test))
                test_loss += test_loss.item()

        print(f'Fold {fold+1}, Epoch {epoch+1}/{epochs}, Training Loss: {running_loss}, Test Loss: {test_loss}')
    ensemble_models.append(model)
ensemble_predictions = []
for model in ensemble_models:
    model.eval()
    with torch.no_grad():
        y_pred_test = model.forward(X_test)
        ensemble_predictions.append(y_pred_test.numpy())
final_predictions = np.mean(ensemble_predictions, axis=0)
y_pred_test = np.argmax(final_predictions, axis=1)
accuracy_test_improved = accuracy_score(y_test, y_pred_test)
precision_test, recall_test, f1_test, _ = precision_recall_fscore_support(y_test, y_pred_test, average='binary')
print(f"Ensemble Test Accuracy: {accuracy_test * 100:.2f}%")
print(f"Ensemble Test Precision: {precision_test:.2f}")
print(f"Ensemble Test Recall: {recall_test:.2f}")
print(f"Ensemble Test F1 Score: {f1_test:.2f}")
plt.figure(figsize=(3, 3))
model_names = ['Base Model', 'Current Model']
test_accuracies = [base_accuracy_test, accuracy_test]
plt.bar(model_names, test_accuracies)
plt.ylabel('Test Accuracy')
plt.title('Comparison of Test Accuracy')
plt.show()
plt.figure(figsize=(3, 3))
# percentage_change = ((accuracy_test - base_accuracy_test) / base_accuracy_test) * 100
# plt.bar(model_names, [0, percentage_change], color=['blue', 'green'])
# plt.ylabel('Percentage Change in Test Accuracy')
# plt.title('Comparison of Test Accuracy Improvement')
# plt.show()

# **PART-III**

In [ ]:
!unzip cnn_dataset.zip

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
from scipy.sparse import load_npz
from pickle import loads
class LoadDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(root_dir)
        self.data = self.load_data()
    def load_data(self):
        data = []
        for class_name in self.classes:
            class_path = os.path.join(self.root_dir, class_name)
            for filename in os.listdir(class_path):
                img_path = os.path.join(class_path, filename)
                data.append((img_path, class_name))
        return data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        label_index = self.classes.index(label)
        label_tensor = torch.tensor(label_index, dtype=torch.long)
        return image, label_tensor
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    # transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
folder = 'cnn_dataset'
dataset = LoadDataset(root_dir=folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
images, labels = next(iter(dataloader))
def image_show(img, title, ax):
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))
    ax.set_title(title)
    ax.axis('off')
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8, 8))
for i in range(3):
    for j in range(3):
        index = i * 3 + j
        image_show(images[index], title=f"Label: {labels[index]}", ax=axes[i, j])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import torchvision
def img_show(img, title):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()
grid = torchvision.utils.make_grid(images, nrow=8, padding=2)
img_show(grid, title="Batch of Images")
print("Labels:", labels)



In [ ]:
from collections import defaultdict

class_distribution = defaultdict(int)
total_samples = len(dataset)
for _, label in dataset:
    class_distribution[label] += 1
print("Class distribution:")
# for class_name, count in class_distribution.items():
#     print(f"{class_name}: {count} samples")
print("\nDataset statistics:")
print(f"Number of samples: {total_samples}")
print(f"Number of classes: {len(dataset.classes)}")
average_samples_per_class = total_samples / len(dataset.classes)
print(f"Average samples per class: {average_samples_per_class:.2f}")
max_class = max(class_distribution, key=class_distribution.get)
min_class = min(class_distribution, key=class_distribution.get)

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(dataset, train_size=0.8, random_state=42)
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
test_dataset, remaining_test_dataset = train_test_split(test_dataset, test_size=0.5, random_state=42)
batch_size = 32
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(remaining_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_images, train_labels = next(iter(train_dataloader))
print("BATCH OF TRAIN IMAGES")
def train_image_show(img, title, ax):
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))
    ax.set_title(title)
    ax.axis('off')
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8, 8))
for i in range(3):
    for j in range(3):
        index = i * 3 + j
        train_image_show(images[index], title=f"Label: {labels[index]}", ax=axes[i, j])
plt.show()

In [ ]:
test_images, test_labels = next(iter(test_dataloader))
print("BATCH OF TEST IMAGES")
def test_image_show(img, title, ax):
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))
    ax.set_title(title)
    ax.axis('off')
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8, 8))
for i in range(3):
    for j in range(3):
        index = i * 3 + j
        test_image_show(images[index], title=f"Label: {labels[index]}", ax=axes[i, j])
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score
import time
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(256 * 3 * 3, 256)
        self.relu4 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(256, 128)
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(128, len(dataset.classes))
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout1(self.relu4(self.fc1(x)))
        x = self.dropout2(self.relu5(self.fc2(x)))
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x

model = CNN()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
train_accuracies = []
val_accuracies = []
test_accuracies = []
train_losses = []
val_losses = []
test_losses = []
f1_scores = []
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  total_loss = 0.0
  correct = 0
  total_samples = 0

  for images, labels in train_dataloader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total_samples += labels.size(0)
      correct += (predicted == labels).sum().item()

  train_accuracy = correct / total_samples
  average_loss = total_loss / len(train_dataloader)
  end_time = time.time()
  training_time = end_time - start_time
  model.eval()
  correct = 0
  total = 0
  val_loss = 0.0
  with torch.no_grad():
      for images, labels in val_dataloader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          val_loss += loss.item()
  val_accuracy = correct / total
  val_loss /= len(val_dataloader)


  print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy * 100:.2f}%")
  print(f"Accuracy on the validation set: {val_accuracy * 100:.2f}%")
  model.eval()
  correct = 0
  total = 0
  test_loss = 0.0
  with torch.no_grad():
      for images, labels in test_dataloader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          test_loss += loss.item()
  test_accuracy = correct / total
  test_loss /= len(test_dataloader)
  print(f"Accuracy on the test set: {test_accuracy * 100:.2f}%")
  train_accuracies.append(train_accuracy)
  val_accuracies.append(val_accuracy)
  test_accuracies.append(test_accuracy)
  train_losses.append(average_loss)
  val_losses.append(val_loss)
  test_losses.append(test_loss)

print(f"Training Time: {training_time}")


In [ ]:
import seaborn as sns
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())


conf_matrix = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


fpr_list = []
tpr_list = []
roc_auc_list = []

for i in range(len(dataset.classes)):
    class_labels = np.array(all_labels) == i
    class_probabilities = np.array(all_predictions)[:, None]
    fpr, tpr, _ = roc_curve(class_labels, class_probabilities[:, 0])
    roc_auc = auc(fpr, tpr)

    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)


plt.figure(figsize=(14, 8))
for i in range(len(dataset.classes)):
    plt.plot(fpr_list[i], tpr_list[i], label=f' (Class {i})')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.grid(True)
plt.show()




In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
fig, axs = plt.subplots(2, figsize=(10, 8))
axs[0].plot(range(num_epochs), train_accuracies, label='Training Accuracy')
axs[0].plot(range(num_epochs), val_accuracies, label='Validation Accuracy')
axs[0].plot(range(num_epochs), test_accuracies, label='Test Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Training, Validation, and Test Accuracy Curves')
axs[0].legend()
axs[0].grid(True)


axs[1].plot(range(num_epochs), train_losses, label='Training Loss')
axs[1].plot(range(num_epochs), val_losses, label='Validation Loss')
axs[1].plot(range(num_epochs), test_losses, label='Test Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Training, Validation, and Test Loss Curves')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
!pip install torchsummary


In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

In [ ]:
torch.save(model.state_dict(), "Part_03_CNN.h5")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score
import time
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(256 * 3 * 3, 256)
        self.relu4 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(256, 128)
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(128, len(dataset.classes))
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout1(self.relu4(self.fc1(x)))
        x = self.dropout2(self.relu5(self.fc2(x)))
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x

model = CNN()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

step_size = 2
gamma = 0.5
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

num_epochs = 5
train_accuracies = []
val_accuracies = []
test_accuracies = []
train_losses = []
val_losses = []
test_losses = []
f1_scores = []
for epoch in range(num_epochs):
  start_time = time.time()
  model.train()
  total_loss = 0.0
  correct = 0
  total_samples = 0

  for images, labels in train_dataloader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total_samples += labels.size(0)
      correct += (predicted == labels).sum().item()

  train_accuracy = correct / total_samples
  average_loss = total_loss / len(train_dataloader)
  end_time = time.time()
  training_time = end_time - start_time
  model.eval()
  correct = 0
  total = 0
  val_loss = 0.0
  with torch.no_grad():
      for images, labels in val_dataloader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          val_loss += loss.item()
  val_accuracy = correct / total
  val_loss /= len(val_dataloader)
  scheduler.step()

  print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Training Accuracy: {train_accuracy * 100:.2f}%")
  print(f"Accuracy on the validation set: {val_accuracy * 100:.2f}%")
  model.eval()
  correct = 0
  total = 0
  test_loss = 0.0
  with torch.no_grad():
      for images, labels in test_dataloader:
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          loss = criterion(outputs, labels)
          test_loss += loss.item()
  test_accuracy = correct / total
  test_loss /= len(test_dataloader)
  print(f"Accuracy on the test set: {test_accuracy * 100:.2f}%")
  train_accuracies.append(train_accuracy)
  val_accuracies.append(val_accuracy)
  test_accuracies.append(test_accuracy)
  train_losses.append(average_loss)
  val_losses.append(val_loss)
  test_losses.append(test_loss)

print(f"Training Time: {training_time}")


In [ ]:
torch.save(model.state_dict(), "Part_03_LR.h5")

In [ ]:
import seaborn as sns
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())


conf_matrix = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


fpr_list = []
tpr_list = []
roc_auc_list = []

for i in range(len(dataset.classes)):
    class_labels = np.array(all_labels) == i
    class_probabilities = np.array(all_predictions)[:, None]
    fpr, tpr, _ = roc_curve(class_labels, class_probabilities[:, 0])
    roc_auc = auc(fpr, tpr)

    fpr_list.append(fpr)
    tpr_list.append(tpr)
    roc_auc_list.append(roc_auc)


plt.figure(figsize=(14, 8))
for i in range(len(dataset.classes)):
    plt.plot(fpr_list[i], tpr_list[i], label=f' (Class {i})')

plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.grid(True)
plt.show()




In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
fig, axs = plt.subplots(2, figsize=(10, 8))
axs[0].plot(range(num_epochs), train_accuracies, label='Training Accuracy')
axs[0].plot(range(num_epochs), val_accuracies, label='Validation Accuracy')
axs[0].plot(range(num_epochs), test_accuracies, label='Test Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].set_title('Training, Validation, and Test Accuracy Curves')
axs[0].legend()
axs[0].grid(True)


axs[1].plot(range(num_epochs), train_losses, label='Training Loss')
axs[1].plot(range(num_epochs), val_losses, label='Validation Loss')
axs[1].plot(range(num_epochs), test_losses, label='Test Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].set_title('Training, Validation, and Test Loss Curves')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
summary(model, input_size=(1, 28, 28))

# **VGG11 IMPLEMENTATION**

In [1]:
import torch
import torch.nn as nn

class VGG11(nn.Module):
    def __init__(self, in_channels=1, num_classes=1000):
        super(VGG11, self).__init__()
        self.features = self._make_layers(in_channels)
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _make_layers(self, in_channels):
        layers = []
        cfg = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, v, kernel_size=3, padding=1),
                           nn.ReLU(inplace=True)]
                in_channels = v

        return nn.Sequential(*layers)
model = VGG11(in_channels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
from torchsummary import summary
summary(model, (1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             640
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define the data directory and transformations
data_dir = 'C:/Users/jcysu/Downloads/cnn_dataset/cnn_dataset/'
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
from sklearn.model_selection import train_test_split
vgg_train_dataset, vgg_test_dataset = train_test_split(dataset, train_size=0.8, random_state=42)
batch_size = 32
vgg_train_dataloader = DataLoader(vgg_train_dataset, batch_size=batch_size, shuffle=True)

In [3]:
test_dataset, validation_dataset = train_test_split(vgg_test_dataset, test_size=0.5, random_state=42)
vgg_test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
vgg_validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

model = VGG11(in_channels=1, num_classes=36)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 5
train_losses = []
val_losses = []
test_losses = []
train_accuracies = []
val_accuracies = []
test_accuracies = []
fpr_list_epoch = []
tpr_list_epoch = []
roc_auc_list_epoch = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in vgg_train_dataloader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(vgg_train_dataloader)
    train_accuracy = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in vgg_validation_dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            val_loss += criterion(outputs, labels).item()

    val_accuracy = correct_val / total_val
    val_losses.append(val_loss / len(vgg_validation_dataloader))
    val_accuracies.append(val_accuracy)

    model.eval()
    correct_test = 0
    total_test = 0
    test_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in vgg_test_dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

            test_loss += criterion(outputs, labels).item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_accuracy = correct_test / total_test
    test_losses.append(test_loss / len(vgg_test_dataloader))
    test_accuracies.append(test_accuracy)
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%, '
          f'Validation Loss: {val_losses[-1]:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%, '
          f'Test Loss: {test_losses[-1]:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%')
    fpr_list = []
    tpr_list = []
    roc_auc_list = []

#     for i in range(36):
    for i in range(len(dataset.classes)):
        class_labels = np.array(all_labels) == i
        class_probabilities = np.array(all_predictions)[:, None]
        fpr, tpr, _ = roc_curve(class_labels, class_probabilities[:, 0])
        roc_auc = auc(fpr, tpr)

        fpr_list.append(fpr)
        tpr_list.append(tpr)
        roc_auc_list.append(roc_auc)
        
#         class_labels = np.array(all_labels) == i
#         class_probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, i].cpu().numpy()
#         fpr, tpr, _ = roc_curve(class_labels, class_probabilities)
#         roc_auc = auc(fpr, tpr)

#         fpr_list.append(fpr)
#         tpr_list.append(tpr)
#         roc_auc_list.append(roc_auc)

    fpr_list_epoch.append(fpr_list)
    tpr_list_epoch.append(tpr_list)
    roc_auc_list_epoch.append(roc_auc_list)


In [ ]:
conf_matrix = confusion_matrix(all_labels, all_predictions)
print(f'Confusion Matrix:\n{conf_matrix}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(36), yticklabels=range(36))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
print(f'Final Training Loss: {train_losses[-1]:.4f}, Final Training Accuracy: {train_accuracies[-1] * 100:.2f}%, '
      f'Final Validation Loss: {val_losses[-1]:.4f}, Final Validation Accuracy: {val_accuracies[-1] * 100:.2f}%, '
      f'Final Test Loss: {test_losses[-1]:.4f}, Final Test Accuracy: {test_accuracies[-1] * 100:.2f}%')

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(num_classes):
    mean_fpr = np.mean([fpr_list_epoch[e][i] for e in range(num_epochs)], axis=0)
    mean_tpr = np.mean([tpr_list_epoch[e][i] for e in range(num_epochs)], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, label=f'Label {i} (AUC = {mean_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
epochs_range = range(1, num_epochs + 1)

plt.figure(figsize=(12, 6))
plt.plot(epochs_range, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(epochs_range, val_accuracies, label='Validation Accuracy', marker='o')
plt.plot(epochs_range, test_accuracies, label='Test Accuracy', marker='o')

plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
epochs_range = range(1, num_epochs + 1)

plt.figure(figsize=(12, 6))
plt.plot(epochs_range, train_losses, label='Train Loss', marker='o')
plt.plot(epochs_range, val_losses, label='Validation Loss', marker='o')
plt.plot(epochs_range, test_losses, label='Test Loss', marker='o')

plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()